# Mosquito Larvae Classification with EfficientNetB2
This notebook uses a pretrained EfficientNetB2 model with transfer learning to classify mosquito larvae species (Aedes aegypti vs Culex quinquefasciatus).

In [3]:
CONFIG = {
    # Data
    "data_path": "MLMI-2024-organized",
    "img_size": 260,  # EfficientNetB2 uses 260x260
    "batch_size": 16,  # Smaller batch size for larger model
    "augment": True,

    # Training
    "epochs": 10,
    "lr": 1e-4,  # Lower learning rate for fine-tuning
    "optimizer": "adam",  # adam | sgd
    "weight_decay": 1e-4,
    "label_smoothing": 0.1,

    # Model
    "freeze_backbone": False,  # Set to True to freeze backbone initially
    "dropout": 0.3,

    # Evaluation
    "use_gradcam": True,
    "output_dir": "experiments/efficientnetb2_run_1",
}

In [4]:
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import os

def get_transforms(cfg, train=True):
    """
    Get transforms for EfficientNetB2
    Uses ImageNet normalization
    """
    if train and cfg["augment"]:
        t = transforms.Compose([
            transforms.Resize((cfg["img_size"], cfg["img_size"])),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.RandomRotation(20),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
            transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    else:
        t = transforms.Compose([
            transforms.Resize((cfg["img_size"], cfg["img_size"])),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    return t

def get_dataloaders(cfg):
    """
    Creates DataLoaders for train and test sets using the structure:
    dataset/
      train/
      test/
    """
    train_transform = get_transforms(cfg, train=True)
    test_transform = get_transforms(cfg, train=False)

    # Use data path from config
    base_path = cfg.get("data_path", "dataset")
    train_dir = os.path.join(base_path, "train")
    test_dir = os.path.join(base_path, "test")

    # Check if directories exist
    if not os.path.exists(train_dir):
        print(f"WARNING: Train directory not found at {train_dir}")
        print("Please ensure 'dataset/train' and 'dataset/test' exist.")
    
    try:
        train_ds = datasets.ImageFolder(train_dir, train_transform)
        test_ds = datasets.ImageFolder(test_dir, test_transform)

        print(f"Train dataset: {len(train_ds)} images")
        print(f"Test dataset: {len(test_ds)} images")
        print(f"Classes: {train_ds.classes}")

        train_dl = DataLoader(train_ds, batch_size=cfg["batch_size"], shuffle=True, num_workers=2)
        test_dl = DataLoader(test_ds, batch_size=cfg["batch_size"], shuffle=False, num_workers=2)
        
        return train_dl, test_dl
    except Exception as e:
        print(f"Error loading datasets: {e}")
        return None, None

## Data Leakage Check
Before training, we verify that there are no duplicate or near-duplicate images between train and test sets. This checks for:
1. **Exact duplicates** - using MD5 hash
2. **Near duplicates** - using perceptual hashing
3. **Filename overlaps** - suspicious naming patterns

⚠️ **Note**: You'll need to install `imagehash` if not already installed: `pip install imagehash`

In [ ]:
import hashlib
from pathlib import Path
from collections import defaultdict
import imagehash
from PIL import Image

def check_data_leakage(data_path, hash_size=8):
    """
    Check for data leakage between train and test sets.
    
    Args:
        data_path: Path to dataset root (containing train/ and test/)
        hash_size: Size for perceptual hash (default 8)
    
    Returns:
        dict with leakage statistics
    """
    print("="*70)
    print("DATA LEAKAGE CHECK")
    print("="*70)
    
    base_path = Path(data_path)
    train_dir = base_path / "train"
    test_dir = base_path / "test"
    
    if not train_dir.exists() or not test_dir.exists():
        print("ERROR: Train or test directory not found!")
        return None
    
    # 1. Check for exact duplicates using MD5 hash
    print("\n1. Checking for EXACT duplicates (MD5 hash)...")
    
    def compute_md5(file_path):
        """Compute MD5 hash of file"""
        hash_md5 = hashlib.md5()
        with open(file_path, "rb") as f:
            for chunk in iter(lambda: f.read(4096), b""):
                hash_md5.update(chunk)
        return hash_md5.hexdigest()
    
    # Get all image files
    train_files = list(train_dir.rglob("*.jpg")) + list(train_dir.rglob("*.png"))
    test_files = list(test_dir.rglob("*.jpg")) + list(test_dir.rglob("*.png"))
    
    print(f"   Train images: {len(train_files)}")
    print(f"   Test images: {len(test_files)}")
    
    # Compute hashes
    print("   Computing MD5 hashes for train set...")
    train_hashes = {compute_md5(f): f for f in train_files}
    
    print("   Computing MD5 hashes for test set...")
    test_hashes = {compute_md5(f): f for f in test_files}
    
    # Find exact matches
    exact_duplicates = set(train_hashes.keys()) & set(test_hashes.keys())
    
    if exact_duplicates:
        print(f"\n   ⚠️  WARNING: Found {len(exact_duplicates)} EXACT duplicate(s)!")
        for i, hash_val in enumerate(list(exact_duplicates)[:5], 1):
            print(f"      {i}. Train: {train_hashes[hash_val].name}")
            print(f"         Test:  {test_hashes[hash_val].name}")
        if len(exact_duplicates) > 5:
            print(f"      ... and {len(exact_duplicates) - 5} more")
    else:
        print("   ✓ No exact duplicates found!")
    
    # 2. Check for near-duplicates using perceptual hash
    print(f"\n2. Checking for NEAR duplicates (perceptual hash, size={hash_size})...")
    print("   This may take a moment...")
    
    def compute_perceptual_hash(file_path, hash_size=8):
        """Compute perceptual hash using average hash"""
        try:
            img = Image.open(file_path)
            return imagehash.average_hash(img, hash_size=hash_size)
        except Exception as e:
            print(f"   Error processing {file_path}: {e}")
            return None
    
    # Compute perceptual hashes
    train_phashes = {}
    for f in train_files:
        h = compute_perceptual_hash(f, hash_size)
        if h is not None:
            train_phashes[f] = h
    
    test_phashes = {}
    for f in test_files:
        h = compute_perceptual_hash(f, hash_size)
        if h is not None:
            test_phashes[f] = h
    
    # Find similar images (hamming distance <= 5)
    similar_pairs = []
    threshold = 5  # Hamming distance threshold
    
    for test_file, test_hash in test_phashes.items():
        for train_file, train_hash in train_phashes.items():
            distance = test_hash - train_hash
            if distance <= threshold:
                similar_pairs.append((train_file, test_file, distance))
    
    if similar_pairs:
        print(f"\n   ⚠️  WARNING: Found {len(similar_pairs)} near-duplicate pair(s)!")
        print(f"   (Hamming distance <= {threshold})")
        for i, (train_f, test_f, dist) in enumerate(similar_pairs[:5], 1):
            print(f"      {i}. Distance={dist}")
            print(f"         Train: {train_f.name}")
            print(f"         Test:  {test_f.name}")
        if len(similar_pairs) > 5:
            print(f"      ... and {len(similar_pairs) - 5} more")
    else:
        print(f"   ✓ No near duplicates found (threshold={threshold})!")
    
    # 3. Check filename patterns
    print("\n3. Checking for suspicious filename overlaps...")
    train_names = set(f.name for f in train_files)
    test_names = set(f.name for f in test_files)
    
    name_overlaps = train_names & test_names
    if name_overlaps:
        print(f"   ⚠️  WARNING: {len(name_overlaps)} files with same name in train and test!")
        for i, name in enumerate(list(name_overlaps)[:5], 1):
            print(f"      {i}. {name}")
        if len(name_overlaps) > 5:
            print(f"      ... and {len(name_overlaps) - 5} more")
    else:
        print("   ✓ No filename overlaps found!")
    
    # Summary
    print("\n" + "="*70)
    print("SUMMARY")
    print("="*70)
    leakage_found = exact_duplicates or similar_pairs or name_overlaps
    
    if leakage_found:
        print("❌ POTENTIAL DATA LEAKAGE DETECTED!")
        print(f"   - Exact duplicates: {len(exact_duplicates)}")
        print(f"   - Near duplicates: {len(similar_pairs)}")
        print(f"   - Filename overlaps: {len(name_overlaps)}")
        print("\n⚠️  Please review and fix the data split before training!")
    else:
        print("✓ NO DATA LEAKAGE DETECTED")
        print("  Train and test sets are properly separated.")
    
    print("="*70)
    
    return {
        "exact_duplicates": len(exact_duplicates),
        "near_duplicates": len(similar_pairs),
        "filename_overlaps": len(name_overlaps),
        "leakage_detected": leakage_found
    }

# Run the data leakage check
leakage_results = check_data_leakage(CONFIG["data_path"])

## Investigate Near-Duplicates
If near-duplicates were detected, use the cells below to investigate them further.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
import imagehash

def visualize_duplicate_pairs(data_path, num_pairs=5, max_distance=5):
    """
    Visualize near-duplicate pairs to manually inspect them.
    
    Args:
        data_path: Path to dataset root
        num_pairs: Number of pairs to visualize
        max_distance: Maximum hamming distance to consider
    """
    base_path = Path(data_path)
    train_dir = base_path / "train"
    test_dir = base_path / "test"
    
    # Get image files
    train_files = list(train_dir.rglob("*.jpg")) + list(train_dir.rglob("*.png"))
    test_files = list(test_dir.rglob("*.jpg")) + list(test_dir.rglob("*.png"))
    
    print(f"Finding near-duplicate pairs (distance <= {max_distance})...")
    
    # Compute hashes
    train_hashes = {}
    for f in train_files[:100]:  # Limit to first 100 for quick check
        try:
            img = Image.open(f)
            train_hashes[f] = imagehash.average_hash(img, hash_size=8)
        except:
            pass
    
    test_hashes = {}
    for f in test_files[:50]:  # Limit to first 50 for quick check
        try:
            img = Image.open(f)
            test_hashes[f] = imagehash.average_hash(img, hash_size=8)
        except:
            pass
    
    # Find pairs
    pairs = []
    for test_file, test_hash in test_hashes.items():
        for train_file, train_hash in train_hashes.items():
            distance = test_hash - train_hash
            if distance <= max_distance:
                pairs.append((train_file, test_file, distance))
                if len(pairs) >= num_pairs:
                    break
        if len(pairs) >= num_pairs:
            break
    
    # Visualize
    if not pairs:
        print("No pairs found in sample!")
        return
    
    fig, axes = plt.subplots(len(pairs), 2, figsize=(10, 4*len(pairs)))
    if len(pairs) == 1:
        axes = [axes]
    
    for idx, (train_f, test_f, dist) in enumerate(pairs):
        # Load images
        train_img = Image.open(train_f)
        test_img = Image.open(test_f)
        
        # Plot train
        axes[idx][0].imshow(train_img)
        axes[idx][0].set_title(f"TRAIN: {train_f.parent.name}/{train_f.name}\nDistance: {dist}", fontsize=8)
        axes[idx][0].axis('off')
        
        # Plot test
        axes[idx][1].imshow(test_img)
        axes[idx][1].set_title(f"TEST: {test_f.parent.name}/{test_f.name}", fontsize=8)
        axes[idx][1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nInspect the pairs above:")
    print("- If they look like the SAME mosquito specimen → TRUE LEAKAGE")
    print("- If they're DIFFERENT mosquitoes but similar → FALSE POSITIVE (OK)")

# Visualize some duplicate pairs
visualize_duplicate_pairs(CONFIG["data_path"], num_pairs=5, max_distance=5)

In [ ]:
def analyze_leakage_statistics(data_path, distance_threshold=5):
    """
    Provide detailed statistics about potential duplicates.
    """
    from pathlib import Path
    import imagehash
    from PIL import Image
    from collections import defaultdict
    
    base_path = Path(data_path)
    train_dir = base_path / "train"
    test_dir = base_path / "test"
    
    print("="*70)
    print("DETAILED LEAKAGE ANALYSIS")
    print("="*70)
    
    # Get files
    train_files = list(train_dir.rglob("*.jpg")) + list(train_dir.rglob("*.png"))
    test_files = list(test_dir.rglob("*.jpg")) + list(test_dir.rglob("*.png"))
    
    print(f"\n📊 Dataset Statistics:")
    print(f"   Train images: {len(train_files)}")
    print(f"   Test images: {len(test_files)}")
    print(f"   Max possible pairs: {len(train_files) * len(test_files):,}")
    
    # Compute hashes (sample for speed)
    print("\n🔍 Computing perceptual hashes (sampling)...")
    sample_size = min(200, len(train_files))
    train_sample = train_files[:sample_size]
    test_sample = test_files[:min(100, len(test_files))]
    
    train_hashes = {}
    for f in train_sample:
        try:
            img = Image.open(f)
            train_hashes[f] = imagehash.average_hash(img, hash_size=8)
        except:
            pass
    
    test_hashes = {}
    for f in test_sample:
        try:
            img = Image.open(f)
            test_hashes[f] = imagehash.average_hash(img, hash_size=8)
        except:
            pass
    
    # Analyze distances
    distances = []
    for test_hash in list(test_hashes.values())[:20]:
        for train_hash in list(train_hashes.values())[:20]:
            distances.append(test_hash - train_hash)
    
    if distances:
        print(f"\n📈 Distance Distribution (sample of {len(distances)} pairs):")
        print(f"   Min distance: {min(distances)}")
        print(f"   Max distance: {max(distances)}")
        print(f"   Mean distance: {sum(distances)/len(distances):.2f}")
        print(f"   Median distance: {sorted(distances)[len(distances)//2]}")
        
        # Count by threshold
        for thresh in [3, 5, 8, 10]:
            count = sum(1 for d in distances if d <= thresh)
            pct = (count / len(distances)) * 100
            print(f"   Pairs with distance ≤ {thresh:2d}: {count:4d} ({pct:5.1f}%)")
    
    print("\n💡 RECOMMENDATIONS:")
    
    if len(distances) > 0:
        similar_pct = (sum(1 for d in distances if d <= 5) / len(distances)) * 100
        
        if similar_pct > 50:
            print("   ⚠️  HIGH similarity detected (>50% of pairs have distance ≤ 5)")
            print("   This is COMMON for biological images of the same species!")
            print("\n   Options:")
            print("   1. INCREASE threshold to 3 or less for stricter checking")
            print("   2. Manually inspect pairs using visualization above")
            print("   3. If images are different individuals → PROCEED with training")
            print("   4. Check if same specimen appears in both sets → FIX split")
        elif similar_pct > 20:
            print("   ⚠️  MODERATE similarity (20-50% of pairs)")
            print("   Recommend manual inspection of flagged pairs")
        else:
            print("   ✓ LOW similarity detected (<20% of pairs)")
            print("   Most flagged pairs are likely different specimens")
    
    print("\n" + "="*70)

# Run detailed analysis
analyze_leakage_statistics(CONFIG["data_path"], distance_threshold=5)

In [5]:
import torch.nn as nn
from torchvision import models

class EfficientNetB2Classifier(nn.Module):
    def __init__(self, cfg, num_classes=2):
        super().__init__()
        
        # Load pretrained EfficientNetB2
        self.model = models.efficientnet_b2(weights=models.EfficientNet_B2_Weights.IMAGENET1K_V1)
        
        # Freeze backbone if specified
        if cfg.get("freeze_backbone", False):
            for param in self.model.features.parameters():
                param.requires_grad = False
            print("Backbone frozen - only training classifier head")
        else:
            print("Training full model (backbone + classifier)")
        
        # Get the number of input features for the classifier
        in_features = self.model.classifier[1].in_features
        
        # Replace classifier with custom head
        self.model.classifier = nn.Sequential(
            nn.Dropout(p=cfg["dropout"], inplace=True),
            nn.Linear(in_features, num_classes)
        )
    
    def forward(self, x):
        return self.model(x)
    
    def unfreeze_backbone(self):
        """Unfreeze the backbone for fine-tuning"""
        for param in self.model.features.parameters():
            param.requires_grad = True
        print("Backbone unfrozen - now training full model")

In [6]:
from sklearn.metrics import roc_auc_score, classification_report
import torch
import numpy as np

def evaluate(model, loader, device):
    model.eval()
    y_true, y_score = [], []
    y_pred = []

    print("\nRunning Evaluation...")
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            # Forward pass
            logits = model(x)
            # Get probabilities
            probs = logits.softmax(1)[:, 1].cpu()
            # Get hard predictions
            preds = logits.argmax(1).cpu()

            y_true.extend(y.numpy())
            y_score.extend(probs.numpy())
            y_pred.extend(preds.numpy())

    # Check if we have gathered any data
    if not y_true:
        print("No evaluation data found.")
        return

    try:
        auc = roc_auc_score(y_true, y_score)
        print(f"ROC-AUC: {auc:.4f}")
    except ValueError:
        print("ROC-AUC: N/A (Only one class present in targets)")

    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, digits=4))

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc
import numpy as np
import os
import torch
import cv2

def plot_training_curves(train_losses, train_accs, output_dir):
    plt.figure(figsize=(10, 5))
    
    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.title('Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(train_accs, label='Train Accuracy', color='orange')
    plt.title('Training Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'training_curves.png'), dpi=150)
    plt.close()

def plot_confusion_matrix(y_true, y_pred, classes, output_dir):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'confusion_matrix.png'), dpi=150)
    plt.close()

def plot_roc_curve(y_true, y_scores, output_dir):
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.grid(True, alpha=0.3)
    plt.savefig(os.path.join(output_dir, 'roc_curve.png'), dpi=150)
    plt.close()

def apply_gradcam(model, image_tensor, target_class=None):
    """
    Generates GradCAM heatmap for EfficientNetB2.
    image_tensor: (1, C, H, W)
    """
    model.eval()
    
    # Hook to capture gradients and activations
    gradients = []
    activations = []
    
    def backward_hook(module, grad_input, grad_output):
        gradients.append(grad_output[0])
        
    def forward_hook(module, input, output):
        activations.append(output)
        
    # Hook the last convolutional layer in EfficientNet features
    # For EfficientNetB2, the features are in model.model.features
    target_layer = model.model.features[-1]
    
    handle_f = target_layer.register_forward_hook(forward_hook)
    handle_b = target_layer.register_full_backward_hook(backward_hook)
    
    # Forward
    output = model(image_tensor)
    if target_class is None:
        target_class = output.argmax(dim=1).item()
        
    # Backward
    model.zero_grad()
    score = output[0, target_class]
    score.backward()
    
    # Get captures
    grads = gradients[0].cpu().data.numpy()[0] # (C, H, W)
    fmaps = activations[0].cpu().data.numpy()[0] # (C, H, W)
    
    handle_f.remove()
    handle_b.remove()
    
    # Weights = Global Average Pooling of Gradients
    weights = np.mean(grads, axis=(1, 2)) # (C,)
    
    # Weighted combination of feature maps
    cam = np.zeros(fmaps.shape[1:], dtype=np.float32) # (H, W)
    for i, w in enumerate(weights):
        cam += w * fmaps[i]
        
    # ReLU
    cam = np.maximum(cam, 0)
    
    # Normalize
    if np.max(cam) > 0:
        cam = cam / np.max(cam)
    else:
        cam = cam # avoid div by zero
        
    return cam

def denormalize_image(img_tensor):
    """Denormalize ImageNet normalized image"""
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    
    img = img_tensor.cpu().numpy().transpose(1, 2, 0)
    img = std * img + mean
    img = np.clip(img, 0, 1)
    return img

def save_gradcam_image(image_tensor, cam, output_path, label_text):
    # Denormalize image for visualization
    img = denormalize_image(image_tensor)
    img = np.uint8(255 * img)
    
    # Resize cam to image size
    heatmap = cv2.resize(cam, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    
    # Superimpose
    superimposed_img = heatmap * 0.4 + img * 0.6
    superimposed_img = np.clip(superimposed_img, 0, 255).astype(np.uint8)
    
    # Add label
    cv2.putText(superimposed_img, label_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    cv2.imwrite(output_path, cv2.cvtColor(superimposed_img, cv2.COLOR_RGB2BGR))

In [8]:
import torch
import torch.nn as nn
from torch.optim import Adam, SGD
import time
import os
import random

def get_optimizer(cfg, model):
    if cfg["optimizer"] == "adam":
        print(f"Using Adam optimizer with lr={cfg['lr']}, weight_decay={cfg['weight_decay']}")
        return Adam(model.parameters(), lr=cfg["lr"], weight_decay=cfg["weight_decay"])
    print(f"Using SGD optimizer with lr={cfg['lr']}, momentum=0.9, weight_decay={cfg['weight_decay']}")
    return SGD(model.parameters(), lr=cfg["lr"], momentum=0.9, weight_decay=cfg["weight_decay"])

def train_model():
    # 0. Setup Output Dir
    os.makedirs(CONFIG["output_dir"], exist_ok=True)
    print(f"Outputs will be saved in: {CONFIG['output_dir']}")

    # 1. Setup
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Running on: {DEVICE}")
    
    # 2. Data
    print("Loading data...")
    train_dl, test_dl = get_dataloaders(CONFIG)
    
    if train_dl is None or test_dl is None:
        print("Data loading failed.")
        return

    # 3. Model
    print("Initializing EfficientNetB2 model...")
    model = EfficientNetB2Classifier(CONFIG, num_classes=2).to(DEVICE)
    
    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    
    # 4. Optimization
    optimizer = get_optimizer(CONFIG, model)
    criterion = nn.CrossEntropyLoss(label_smoothing=CONFIG["label_smoothing"])
    
    # Trackers
    train_losses = []
    train_accs = []
    
    # 5. Training Loop
    print(f"\nStarting training for {CONFIG['epochs']} epochs...")
    print("="*60)
    start_time = time.time()
    
    for epoch in range(CONFIG["epochs"]):
        model.train()
        correct, total, loss_sum = 0, 0, 0
        
        for i, (x, y) in enumerate(train_dl):
            x, y = x.to(DEVICE), y.to(DEVICE)
            
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            
            loss_sum += loss.item()
            correct += (out.argmax(1) == y).sum().item()
            total += y.size(0)
            
        # Stats
        epoch_acc = correct / total if total > 0 else 0
        avg_loss = loss_sum / len(train_dl) if len(train_dl) > 0 else 0
        
        train_losses.append(avg_loss)
        train_accs.append(epoch_acc)
        
        print(f"[Epoch {epoch+1:02d}/{CONFIG['epochs']}] Loss={avg_loss:.4f} Acc={epoch_acc:.4f}")
        
    total_time = time.time() - start_time
    print("="*60)
    print(f"Training finished in {total_time:.2f}s ({total_time/60:.2f} minutes)")
    
    # Save training curves
    print("\nGenerating training curves...")
    plot_training_curves(train_losses, train_accs, CONFIG["output_dir"])
    
    # 6. Evaluation
    print("\nEvaluating on Test Set...")
    model.eval()
    y_true, y_score, y_pred = [], [], []
    
    with torch.no_grad():
        for x, y in test_dl:
            x = x.to(DEVICE)
            logits = model(x)
            probs = logits.softmax(1)[:, 1].cpu()
            preds = logits.argmax(1).cpu()

            y_true.extend(y.numpy())
            y_score.extend(probs.numpy())
            y_pred.extend(preds.numpy())
            
    # Save Model
    save_path = os.path.join(CONFIG["output_dir"], "efficientnetb2_mosq.pth")
    torch.save(model.state_dict(), save_path)
    print(f"\nModel saved to {save_path}")
    
    # Generate Plots
    print("Generating ROC Curve...")
    plot_roc_curve(y_true, y_score, CONFIG["output_dir"])
    
    print("Generating Confusion Matrix...")
    classes = ["Aedes aegypti", "Culex quinquefasciatus"]
    plot_confusion_matrix(y_true, y_pred, classes, CONFIG["output_dir"])
    
    # 7. GradCAM Generation
    if CONFIG["use_gradcam"]:
        print("\nGenerating GradCAM samples...")
        model.eval()
        gradcam_dir = os.path.join(CONFIG["output_dir"], "gradcam")
        os.makedirs(gradcam_dir, exist_ok=True)
        
        # Pick 5 random images from test set
        inputs, labels = next(iter(test_dl))
        indices = random.sample(range(len(inputs)), min(5, len(inputs)))
        
        for idx in indices:
            img_tensor = inputs[idx].unsqueeze(0).to(DEVICE) # (1, 3, H, W)
            label = labels[idx].item()
            class_name = classes[label]
            
            # Predict
            out = model(img_tensor)
            pred = out.argmax(1).item()
            pred_name = classes[pred]
            
            # Generate CAM
            cam = apply_gradcam(model, img_tensor, target_class=pred)
            
            # Save
            fname = f"cam_{idx}_true_{class_name}_pred_{pred_name}.jpg"
            save_path = os.path.join(gradcam_dir, fname)
            label_text = f"True: {class_name}, Pred: {pred_name}"
            save_gradcam_image(img_tensor[0], cam, save_path, label_text)
            print(f"  Saved {fname}")

    # Text Evaluation
    evaluate(model, test_dl, DEVICE)
    
    print("\n" + "="*60)
    print("Training and Evaluation Complete!")
    print("="*60)

## Run Training
Execute the cell below to start the training process with EfficientNetB2.

In [9]:
# Run the training
train_model()

Outputs will be saved in: experiments/efficientnetb2_run_1
Running on: cuda
Loading data...
Train dataset: 560 images
Test dataset: 240 images
Classes: ['Aedes aegypti', 'Culex quinquefasciatus']
Initializing EfficientNetB2 model...
Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to C:\Users\tamji/.cache\torch\hub\checkpoints\efficientnet_b2_rwightman-c35c1473.pth


100%|██████████| 35.2M/35.2M [00:03<00:00, 9.63MB/s]


Training full model (backbone + classifier)
Total parameters: 7,703,812
Trainable parameters: 7,703,812
Using Adam optimizer with lr=0.0001, weight_decay=0.0001

Starting training for 10 epochs...
[Epoch 01/10] Loss=0.5480 Acc=0.7893
[Epoch 02/10] Loss=0.3201 Acc=0.9393
[Epoch 03/10] Loss=0.2925 Acc=0.9607
[Epoch 04/10] Loss=0.2561 Acc=0.9786
[Epoch 05/10] Loss=0.2452 Acc=0.9786
[Epoch 06/10] Loss=0.2482 Acc=0.9786
[Epoch 07/10] Loss=0.2304 Acc=0.9929
[Epoch 08/10] Loss=0.2297 Acc=0.9911
[Epoch 09/10] Loss=0.2223 Acc=0.9964
[Epoch 10/10] Loss=0.2235 Acc=0.9911
Training finished in 741.64s (12.36 minutes)

Generating training curves...

Evaluating on Test Set...

Model saved to experiments/efficientnetb2_run_1\efficientnetb2_mosq.pth
Generating ROC Curve...
Generating Confusion Matrix...

Generating GradCAM samples...
  Saved cam_10_true_Aedes aegypti_pred_Aedes aegypti.jpg
  Saved cam_7_true_Aedes aegypti_pred_Aedes aegypti.jpg
  Saved cam_8_true_Aedes aegypti_pred_Aedes aegypti.jpg
  